In [1]:
import argparse

import torch
import uuid

import numpy as np

from torch_geometric.nn import Node2Vec
from torch_geometric.utils import erdos_renyi_graph

import os
import csv

import gph
import gtda
from gtda import diagrams
from gtda import homology

import matplotlib.pyplot as plt

import sys
# appending a path
sys.path.append('../rawos')
import metrics


In [31]:
def total_persistence(diagram, p=2):
    """Calculate total persistence of a persistence diagram."""
    return np.sum(np.power(np.abs(np.diff(diagram[:, 0:2])), 2))

def get_dimension(diagrams, dim=0):
    """Get specific dimension of persistence diagram."""
    # This reads a little bit weird because of the idiosyncratic way of
    # handling masks. We first extract all triples in all time steps
    # that match the specified dimension. *Then* we remove the dimension
    # information (since it is spurious) and reshape the array to
    # account for the original number of time steps again. Whew!
    T = len(diagrams)
    mask = diagrams[..., 2] == dim
    diagrams = diagrams[mask][:, :2].reshape(T, -1, 2)

    return diagrams, T

def get_dimension(diagrams, dim=0):
    """Get specific dimension of persistence diagram."""
    # This reads a little bit weird because of the idiosyncratic way of
    # handling masks. We first extract all triples in all time steps
    # that match the specified dimension. *Then* we remove the dimension
    # information (since it is spurious) and reshape the array to
    # account for the original number of time steps again. Whew!
    mask = diagrams[..., 2] == dim
    diagrams = diagrams[mask][:, :2]
    return diagrams


def total_persistence_vectors(diagrams, dim=0, p=2):
    """Calculate total persistence of persistence diagrams."""
    diagrams, T = get_dimension(diagrams, dim)

    persistence = np.diff(diagrams)
    persistence = persistence[np.isfinite(persistence)].reshape(T, -1)

    # Ensure that the simplification is only performed over a given
    # axis.
    result = np.sum(np.power(np.abs(persistence), p), axis=1)
    return result


In [48]:
def compute_total_persistence(filename, max_dim): 
    
    Emb_list= []
    dim = 0
    for i,file in zip(range(len(files)),files): 
        emb = np.loadtxt(path+'/'+file, delimiter="\t")
        diam = metrics.diameter(emb)
        Emb_list.append(emb/diam)
        if emb.shape[0] > dim:
            dim = emb.shape[0] ### what does dim do again??


        Embs = np.zeros((len(Emb_list),dim, d)) 
    
    
    
    emb = np.loadtxt(filename, delimiter="\t")
    diam = metrics.diameter(emb)
    emb= emb/diam 
    Embs = np.zeros((1,dim, d)) 
    ph = gtda.homology.VietorisRipsPersistence(metric='euclidean', max_edge_length=np.inf, homology_dimensions=tuple(range(max_dim+1)), coeff=2, infinity_values=None, n_jobs=None).fit_transform(emb)
    
    total_pers=np.zeros((1,max_dim+1))
    for i in range(max_dim+1): 
        dgm_i= get_dimension(ph[0,:,:],i)
        total_pers[i]= total_persistence(dgm_i)
    return total_pers
    
    
    
    

In [49]:
filename ='../results/er-c5-d16-l5-n10-p0_25-1de110b3e022445bbcffe280ac0601a8.tsv' 
compute_total_persistence(filename, 1)

ValueError: Input must be a single 3D array or a list of 2D arrays. Array of dimension 2 passed.
Reshape your input X using X.reshape(1, *X.shape) or X[None, :, :] if X is a single point cloud/distance matrix/adjacency matrix of a weighted graph.

In [43]:
get_dimension(ph[0])

array([[0.        , 0.0973931 ],
       [0.        , 0.09901441],
       [0.        , 0.10474426],
       [0.        , 0.10668171],
       [0.        , 0.10695526],
       [0.        , 0.10865569],
       [0.        , 0.10905732],
       [0.        , 0.10936204],
       [0.        , 0.11315637],
       [0.        , 0.11375583],
       [0.        , 0.11554391],
       [0.        , 0.11800739],
       [0.        , 0.11856474],
       [0.        , 0.11902677],
       [0.        , 0.11992528],
       [0.        , 0.1200401 ],
       [0.        , 0.12008904],
       [0.        , 0.12087532],
       [0.        , 0.12186912],
       [0.        , 0.1222083 ],
       [0.        , 0.12256264],
       [0.        , 0.1249334 ],
       [0.        , 0.12560883],
       [0.        , 0.12748806],
       [0.        , 0.1325743 ],
       [0.        , 0.13285266],
       [0.        , 0.13379018],
       [0.        , 0.13457949],
       [0.        , 0.13597748],
       [0.        , 0.13702971],
       [0.

In [3]:
## import files
c = 5
d = 16
l = 5
n = 10

max_dim = 1

"""c = 5
d= 5
l=10
n=10"""



filename = 'er'
filename += '-c'+str(c)
filename += '-d'+ str(d)
filename += '-l'+str(l)
filename += '-n'+ str(n)

## get filenames according to filter
path = '../results'
#path = '../rawos'
files = []
for i in os.listdir(path):
    if os.path.isfile(os.path.join(path,i)) and filename in i:
        files.append(i)

## load embeddings
Emb_list= []
dim = 0
for i,file in zip(range(len(files)),files): 
    emb = np.loadtxt(path+'/'+file, delimiter="\t")
    diam = metrics.diameter(emb)
    Emb_list.append(emb/diam)
    if emb.shape[0] > dim:
        dim = emb.shape[0]
        

Embs = np.zeros((len(Emb_list),dim, d)) 
for i in range(len(Emb_list)):
    Embs[i,:,:] = Emb_list[i] 

ph = gtda.homology.VietorisRipsPersistence(metric='euclidean', max_edge_length=np.inf, homology_dimensions=tuple(range(max_dim+1)), coeff=2, infinity_values=None, n_jobs=None).fit_transform(Embs)


SyntaxError: positional argument follows keyword argument (<ipython-input-6-721ae9f4ea3f>, line 1)

In [93]:
Total_pers = []
for i in range(ph.shape[0]):
    Total_pers.append(total_persistence(ph[i,:,:],2))
print(Total_pers)

[9.371290405921307, 10.520334260273302, 13.100107757770068, 13.702381461918115, 10.836404870206657, 8.648231650589691, 12.827913595219947, 11.22237666626538, 10.15325338617463, 8.122722488466898]


In [94]:
## write files
